In [183]:
from matplotlib.pyplot import draw, fill
import pyrender
import trimesh
from trimesh import remesh
import pandas as pd
import numpy as np
from trimesh import Trimesh
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
import pickle
import matplotlib.mlab as mlab
import seaborn as sns
import skeletor as sk
from view_data import view_mesh
import copy
from sklearn.preprocessing import MinMaxScaler
from project_statistics import get_outliers
from create_dataset import load_dataset,dir_to_sorted_file_list
from trimesh.repair import fill_holes
from trimesh.points import PointCloud
import ast
import random
from scipy.stats import wasserstein_distance
from random import choice

In [6]:
with open('normalized_final_features_df.pkl', 'rb') as f:
    normalized_data = pickle.load(f)

In [184]:
with open('example_221.pkl', 'rb') as f:
    query = pickle.load(f)

In [185]:
query

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,221,-0.588866,-0.50444,-0.00564,-0.784274,1.0,-0.403296,-0.026324,"[0.12004230565838181, 0.026969857218402962, 0....","[0.2923697357597159, 0.18890070423948155, 0.14...","[0.13017395317108404, 0.06781565377901727, 0.0...","[0.10312794687104994, 0.17491623362292089, 0.1...","[0.2077807427619831, 0.21638811420316606, 0.14...","[0.05692100880800947, 0.22688148311630987, 0.2..."


In [186]:
normalized_data

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,-0.583110,-0.039329,-0.116012,0.230250,1.0,-0.109650,-0.026311,"[0.0015568240788790867, 0.011416709911779968, ...","[0.23664198719878007, 0.22873653163185456, 0.1...","[0.04733241036136359, 0.12355785629727724, 0.1...","[0.09370171953690884, 0.19420534119865968, 0.2...","[0.20435803286582796, 0.15628323200706273, 0.1...","[0.13328919119565, 0.28198800136439334, 0.2750..."
1,1,-0.640930,-0.415322,-0.118042,0.272779,1.0,0.097077,-0.026489,"[0.04924623115577889, 0.054773869346733665, 0....","[0.24772371746019012, 0.22417873330391883, 0.1...","[0.12446351931330472, 0.25626729774176726, 0.1...","[0.151618466968834, 0.300790180899282, 0.27189...","[0.3746741005174281, 0.28757370342144317, 0.16...","[0.07649111547872127, 0.2777064698568048, 0.32..."
2,2,-0.603237,-0.095673,-0.118419,0.280905,1.0,-0.043228,-0.026263,"[0.015625, 0.03780241935483871, 0.137096774193...","[0.25301446542142325, 0.23644242922777522, 0.1...","[0.15300042132295408, 0.21009971309913128, 0.1...","[0.14629938005336757, 0.26599522500652045, 0.2...","[0.29506650883775054, 0.24288265152579097, 0.1...","[0.11953534097064783, 0.3514635956904681, 0.29..."
3,3,-0.520618,0.392970,-0.119893,0.313312,1.0,-0.468747,-0.026398,"[0.011866235167206042, 0.02481121898597627, 0....","[0.20333848283610537, 0.23802740605501274, 0.1...","[0.05069919547378769, 0.09237004193166543, 0.0...","[0.061753907268824106, 0.1599622815641113, 0.1...","[0.1285636899865578, 0.1484260578215597, 0.138...","[0.08151596011476035, 0.2712316674357482, 0.31..."
4,4,-0.724641,-0.475575,-0.095315,-0.114249,1.0,-0.306213,-0.026249,"[0.06380952380952382, 0.10952380952380952, 0.2...","[0.2399117529081428, 0.2335138387484958, 0.155...","[0.17721620537505015, 0.18864821500200563, 0.2...","[0.1078018451664661, 0.2600882470918572, 0.267...","[0.2778780585639791, 0.25130365022061774, 0.16...","[0.07882069795427196, 0.2884476534296029, 0.31..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,-0.365028,-0.011320,-0.113034,0.171236,1.0,-0.081913,-0.026487,"[0.08372530573847602, 0.009877704609595485, 0....","[0.1711191335740072, 0.23331327717609307, 0.20...","[0.10609707180104291, 0.027476935419173686, 0....","[0.10146409947853992, 0.1478740473325311, 0.25...","[0.1829121540312876, 0.11217408744484557, 0.09...","[0.07059767348576013, 0.20198555956678702, 0.2..."
1810,1810,-0.089583,0.813581,-0.138504,0.844492,1.0,-0.125955,-0.026451,"[0.0903030303030303, 0.15575757575757576, 0.38...","[0.16446496687412165, 0.24053402931138326, 0.2...","[0.18598674964866493, 0.2187111021883156, 0.21...","[0.07153182091949407, 0.1783778357759486, 0.24...","[0.13278458140935556, 0.1505320216823931, 0.14...","[0.08239309375627384, 0.27639028307568764, 0.3..."
1811,1811,-0.509943,-0.716180,-0.046532,-0.572259,1.0,-0.403390,-0.026294,"[0.008452192287374538, 0.0306391970417327, 0.0...","[0.2840779153042187, 0.1652189612630143, 0.140...","[0.1258199763285121, 0.4619952255812554, 0.094...","[0.25559188750025075, 0.2389616642259624, 0.12...","[0.4473911211859817, 0.21958314108607996, 0.13...","[0.10658187726935345, 0.2844189452145479, 0.28..."
1812,1812,0.413064,1.070660,-0.142995,1.019764,1.0,-0.306231,-0.026260,"[0.1552892131318395, 0.26003126628452317, 0.28...","[0.23453768531707023, 0.2171845848295785, 0.16...","[0.06275202118482556, 0.09860172126707725, 0.1...","[0.15445262503259977, 0.1181415130298714, 0.18...","[0.29749032038036394, 0.104580014845427, 0.078...","[0.2087588019339178, 0.29097036933015025, 0.20..."


In [10]:
num_samples = 100
descriptor_sample_data = {}

for index, row in normalized_data.iterrows():
    print(index, end='\r')
    while(True):
        samples = random.sample(range(0, 1813), num_samples)
        if (index in samples or len(np.unique(samples))!=len(samples)):
            continue
        else:
            break

    for s in range(len(samples)):
        s_id = samples[s]
        sample_decriptors = normalized_data.loc[s_id][8:]
        
        for feature, column in enumerate(normalized_data.columns[8:]):
            sample_descriptor_feature = sample_decriptors[feature]
            original_descriptor_feature = row[column]
            d = wasserstein_distance(original_descriptor_feature, sample_descriptor_feature)
            
            if column not in descriptor_sample_data.keys():
                descriptor_sample_data[column] = []
            else:
                descriptor_sample_data[column].append(d)

In [142]:
len(descriptor_sample_data['curvature'])

181399

In [146]:
sampled_mean_std = []
sampled_min_max = []
for key in list(descriptor_sample_data.keys()):
    sampled_emd = descriptor_sample_data[key]
    sampled_mean = np.mean(sampled_emd)
    sampled_std = np.std(sampled_emd)
    
    sampled_min = np.min(sampled_emd)
    sampled_max = np.max(sampled_emd)
    
    sampled_mean_std.append((sampled_mean,sampled_std))
    sampled_min_max.append((sampled_min,sampled_max))

In [145]:
sampled_mean_std

[(0.05941732468786712, 0.03227537214801827),
 (0.0272227812635007, 0.016970185823185244),
 (0.052005031097678595, 0.03077671502133887),
 (0.04117402783987219, 0.027071687756029455),
 (0.06223508137347629, 0.03711761044295368),
 (0.03548944494903288, 0.02606357040713323)]

In [147]:
sampled_min_max

[(0.0025589985635132333, 0.20130986568723444),
 (0.0010377887386065889, 0.11379616786167861),
 (0.0026650177725453064, 0.20512993489279302),
 (0.001352901058189793, 0.1923446540048216),
 (0.001312439443033486, 0.20485143010335366),
 (0.0009337864112169531, 0.2009173849151582)]

In [15]:
def standardize(data, idx):
    #sampled_mean_std = np.load('sampled_mean_std.npy')
    #sampled_mean_std = sampled_mean_std
    data_mean = sampled_mean_std[idx][0]  
    data_std = sampled_mean_std[idx][1]  
    normalized_descriptor_data = (data - data_mean)/data_std
    return normalized_descriptor_data

In [23]:
normalized_data.loc[0]

id                                                                0
surface_area                                               -0.58311
volume                                                   -0.0393293
compactness                                               -0.116012
sphericity                                                  0.23025
diameter                                                          1
rectangulairty                                             -0.10965
eccentricity                                             -0.0263109
curvature         [0.0015568240788790867, 0.011416709911779968, ...
A3                [0.23664198719878007, 0.22873653163185456, 0.1...
D1                [0.04733241036136359, 0.12355785629727724, 0.1...
D2                [0.09370171953690884, 0.19420534119865968, 0.2...
D3                [0.20435803286582796, 0.15628323200706273, 0.1...
D4                [0.13328919119565, 0.28198800136439334, 0.2750...
Name: 0, dtype: object

In [191]:
query

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,221,-0.588866,-0.50444,-0.00564,-0.784274,1.0,-0.403296,-0.026324,"[0.12004230565838181, 0.026969857218402962, 0....","[0.2923697357597159, 0.18890070423948155, 0.14...","[0.13017395317108404, 0.06781565377901727, 0.0...","[0.10312794687104994, 0.17491623362292089, 0.1...","[0.2077807427619831, 0.21638811420316606, 0.14...","[0.05692100880800947, 0.22688148311630987, 0.2..."


In [108]:
normalized_data

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,-0.583110,-0.039329,-0.116012,0.230250,1.0,-0.109650,-0.026311,"[0.0015568240788790867, 0.011416709911779968, ...","[0.23664198719878007, 0.22873653163185456, 0.1...","[0.04733241036136359, 0.12355785629727724, 0.1...","[0.09370171953690884, 0.19420534119865968, 0.2...","[0.20435803286582796, 0.15628323200706273, 0.1...","[0.13328919119565, 0.28198800136439334, 0.2750..."
1,1,-0.640930,-0.415322,-0.118042,0.272779,1.0,0.097077,-0.026489,"[0.04924623115577889, 0.054773869346733665, 0....","[0.24772371746019012, 0.22417873330391883, 0.1...","[0.12446351931330472, 0.25626729774176726, 0.1...","[0.151618466968834, 0.300790180899282, 0.27189...","[0.3746741005174281, 0.28757370342144317, 0.16...","[0.07649111547872127, 0.2777064698568048, 0.32..."
2,2,-0.603237,-0.095673,-0.118419,0.280905,1.0,-0.043228,-0.026263,"[0.015625, 0.03780241935483871, 0.137096774193...","[0.25301446542142325, 0.23644242922777522, 0.1...","[0.15300042132295408, 0.21009971309913128, 0.1...","[0.14629938005336757, 0.26599522500652045, 0.2...","[0.29506650883775054, 0.24288265152579097, 0.1...","[0.11953534097064783, 0.3514635956904681, 0.29..."
3,3,-0.520618,0.392970,-0.119893,0.313312,1.0,-0.468747,-0.026398,"[0.011866235167206042, 0.02481121898597627, 0....","[0.20333848283610537, 0.23802740605501274, 0.1...","[0.05069919547378769, 0.09237004193166543, 0.0...","[0.061753907268824106, 0.1599622815641113, 0.1...","[0.1285636899865578, 0.1484260578215597, 0.138...","[0.08151596011476035, 0.2712316674357482, 0.31..."
4,4,-0.724641,-0.475575,-0.095315,-0.114249,1.0,-0.306213,-0.026249,"[0.06380952380952382, 0.10952380952380952, 0.2...","[0.2399117529081428, 0.2335138387484958, 0.155...","[0.17721620537505015, 0.18864821500200563, 0.2...","[0.1078018451664661, 0.2600882470918572, 0.267...","[0.2778780585639791, 0.25130365022061774, 0.16...","[0.07882069795427196, 0.2884476534296029, 0.31..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,-0.365028,-0.011320,-0.113034,0.171236,1.0,-0.081913,-0.026487,"[0.08372530573847602, 0.009877704609595485, 0....","[0.1711191335740072, 0.23331327717609307, 0.20...","[0.10609707180104291, 0.027476935419173686, 0....","[0.10146409947853992, 0.1478740473325311, 0.25...","[0.1829121540312876, 0.11217408744484557, 0.09...","[0.07059767348576013, 0.20198555956678702, 0.2..."
1810,1810,-0.089583,0.813581,-0.138504,0.844492,1.0,-0.125955,-0.026451,"[0.0903030303030303, 0.15575757575757576, 0.38...","[0.16446496687412165, 0.24053402931138326, 0.2...","[0.18598674964866493, 0.2187111021883156, 0.21...","[0.07153182091949407, 0.1783778357759486, 0.24...","[0.13278458140935556, 0.1505320216823931, 0.14...","[0.08239309375627384, 0.27639028307568764, 0.3..."
1811,1811,-0.509943,-0.716180,-0.046532,-0.572259,1.0,-0.403390,-0.026294,"[0.008452192287374538, 0.0306391970417327, 0.0...","[0.2840779153042187, 0.1652189612630143, 0.140...","[0.1258199763285121, 0.4619952255812554, 0.094...","[0.25559188750025075, 0.2389616642259624, 0.12...","[0.4473911211859817, 0.21958314108607996, 0.13...","[0.10658187726935345, 0.2844189452145479, 0.28..."
1812,1812,0.413064,1.070660,-0.142995,1.019764,1.0,-0.306231,-0.026260,"[0.1552892131318395, 0.26003126628452317, 0.28...","[0.23453768531707023, 0.2171845848295785, 0.16...","[0.06275202118482556, 0.09860172126707725, 0.1...","[0.15445262503259977, 0.1181415130298714, 0.18...","[0.29749032038036394, 0.104580014845427, 0.078...","[0.2087588019339178, 0.29097036933015025, 0.20..."


In [192]:
from sklearn.preprocessing import MinMaxScaler

comaprsion_df = normalized_data.copy()
scalar_query_features = np.array(query.loc[0][1:8], dtype = 'float32')
descriptors_query_features = np.vstack(np.array(query.loc[0][8:]))
comaprsion_values = {}
for index, row in normalized_data.iterrows():
#     if (index<=699):
#         continue
    #print("id", index)
    scalar_features = np.array(row[1:8], dtype = 'float32')
    descriptor_features = np.vstack(np.array(row[8:]))
    scalar_distance = distance.cosine(scalar_query_features,scalar_features)
    #print("scalar",scalar_distance)
    total_descriptor_distance = 0
    for i in range(len(descriptors_query_features)):
        descriptor_query_feature = descriptors_query_features[i]
        descriptor_feature = descriptor_features[i]
        descriptor_distance = wasserstein_distance(descriptor_query_feature,descriptor_feature)
        
        descriptor_distance_scaled = (descriptor_distance - sampled_min_max[i][0]) / (sampled_min_max[i][1] - sampled_min_max[i][0])
        #print(descriptor_distance_std)
        #descriptor_distance_scaled = descriptor_distance_std * (sampled_min_max[i][1] - sampled_min_max[i][0]) + sampled_min_max[i][0]
        
        #print(descriptor_distance, descriptor_distance_scaled)
        #print(descriptor_distance_scaled)
        total_descriptor_distance += descriptor_distance
        #total_descriptor_distance = total_descriptor_distance/6
    #print("descriptor",total_descriptor_distance)
    #total_distance = scalar_distance + total_descriptor_distance
    total_distance = scalar_distance 
    comaprsion_values[index] = total_distance

In [190]:
comaprsion_values
sorted(comaprsion_values.items(), key=lambda x: x[1])

[(1236, 0.1515247461561073),
 (105, 0.15818204846748488),
 (229, 0.158612752279846),
 (86, 0.17109070228413484),
 (665, 0.1802504211641865),
 (156, 0.2225676690550199),
 (121, 0.22361953384814187),
 (67, 0.23131672174885806),
 (160, 0.23328948757940493),
 (72, 0.23567204197376018),
 (145, 0.24023482991728937),
 (71, 0.24085137393064024),
 (685, 0.2412544628602423),
 (142, 0.24401290034923806),
 (108, 0.24693304327203816),
 (123, 0.25852584182433314),
 (69, 0.26709541735418163),
 (61, 0.2722410287196246),
 (70, 0.2736306563833878),
 (77, 0.27876126946200225),
 (53, 0.28816413748363534),
 (657, 0.2896926731956796),
 (64, 0.2906461340991231),
 (55, 0.2910852123722421),
 (128, 0.2932795459455292),
 (137, 0.2952469526902898),
 (135, 0.29613219493835563),
 (148, 0.29704186363530083),
 (76, 0.29728270331891304),
 (651, 0.2981263240082316),
 (87, 0.303609622685758),
 (104, 0.3041600743683758),
 (118, 0.3055240531273752),
 (54, 0.31024005886402495),
 (168, 0.3136690446308187),
 (140, 0.31459650

In [193]:
comaprsion_values
sorted(comaprsion_values.items(), key=lambda x: x[1]) # without descriptor

[(238, 0.0014781355857849121),
 (1307, 0.0037515759468078613),
 (665, 0.009279072284698486),
 (61, 0.012724339962005615),
 (53, 0.01961374282836914),
 (54, 0.020912110805511475),
 (55, 0.022826075553894043),
 (145, 0.02560102939605713),
 (57, 0.026356875896453857),
 (56, 0.026668906211853027),
 (113, 0.027780532836914062),
 (70, 0.027879953384399414),
 (64, 0.02939021587371826),
 (68, 0.03111976385116577),
 (38, 0.03459101915359497),
 (72, 0.03619492053985596),
 (142, 0.03742825984954834),
 (71, 0.03752303123474121),
 (121, 0.03865766525268555),
 (86, 0.0389707088470459),
 (69, 0.04052358865737915),
 (41, 0.04141765832901001),
 (45, 0.04200059175491333),
 (123, 0.045337975025177),
 (105, 0.046895384788513184),
 (67, 0.04707682132720947),
 (103, 0.04714256525039673),
 (79, 0.04931282997131348),
 (118, 0.049535155296325684),
 (62, 0.049580156803131104),
 (114, 0.051104962825775146),
 (104, 0.051428139209747314),
 (42, 0.051433265209198),
 (126, 0.05202096700668335),
 (47, 0.0527994632720

In [132]:
comaprsion_values
sorted(comaprsion_values.items(), key=lambda x: x[1])

[(700, 0.0),
 (1112, 0.002103269100189209),
 (722, 0.010791242122650146),
 (709, 0.015851914882659912),
 (683, 0.021650314331054688),
 (721, 0.03862500190734863),
 (484, 0.04272705316543579),
 (485, 0.044491589069366455),
 (525, 0.0532686710357666),
 (482, 0.059733688831329346),
 (710, 0.0639810562133789),
 (487, 0.06819713115692139),
 (483, 0.07131671905517578),
 (787, 0.07180321216583252),
 (486, 0.08027011156082153),
 (718, 0.08309125900268555),
 (144, 0.08805930614471436),
 (488, 0.091033935546875),
 (494, 0.09220618009567261),
 (96, 0.09432810544967651),
 (423, 0.09447121620178223),
 (97, 0.09566497802734375),
 (707, 0.09714758396148682),
 (530, 0.09756410121917725),
 (841, 0.09771353006362915),
 (98, 0.09914213418960571),
 (620, 0.10270422697067261),
 (523, 0.10317903757095337),
 (697, 0.10669362545013428),
 (714, 0.10669362545013428),
 (623, 0.11342936754226685),
 (471, 0.11567765474319458),
 (23, 0.12392443418502808),
 (164, 0.12495869398117065),
 (754, 0.1254950761795044),
 (6

In [57]:
comaprsion_df = normalized_data.copy()

for idx, column in enumerate(normalized_data.columns[8:]):
    comaprsion_df[column] = [standardize(wasserstein_distance(row, query[column][0]), idx) for row in normalized_data[column]]

In [58]:
comaprsion_df

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,-0.583110,-0.039329,-0.116012,0.230250,1.0,-0.109650,-0.026311,0.706911,3.070764,-1.057928,-0.226524,1.880901,-0.226722
1,1,-0.640930,-0.415322,-0.118042,0.272779,1.0,0.097077,-0.026489,-0.202784,2.873449,-0.828880,0.420700,0.384816,-0.263937
2,2,-0.603237,-0.095673,-0.118419,0.280905,1.0,-0.043228,-0.026263,0.035338,2.795507,-0.351252,0.163516,0.921000,-0.775330
3,3,-0.520618,0.392970,-0.119893,0.313312,1.0,-0.468747,-0.026398,-0.478161,3.147285,-0.917406,-0.361934,1.740425,-0.259657
4,4,-0.724641,-0.475575,-0.095315,-0.114249,1.0,-0.306213,-0.026249,0.507394,2.988533,-0.355128,-0.083178,1.036771,-0.198301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,-0.365028,-0.011320,-0.113034,0.171236,1.0,-0.081913,-0.026487,-0.539549,3.298097,-0.675248,-0.387057,1.588114,0.908072
1810,1810,-0.089583,0.813581,-0.138504,0.844492,1.0,-0.125955,-0.026451,-0.624279,3.193066,0.234825,0.118912,2.135730,-0.205266
1811,1811,-0.509943,-0.716180,-0.046532,-0.572259,1.0,-0.403390,-0.026294,0.307365,2.919810,0.003639,-0.285808,-0.104959,-0.092936
1812,1812,0.413064,1.070660,-0.142995,1.019764,1.0,-0.306231,-0.026260,0.174479,3.067702,-1.141324,-0.664696,0.904675,0.120963


In [62]:
query

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,700,-0.88173,-0.955698,-0.121938,0.360174,1.0,0.919625,-0.025173,"[0.03635432667690732, 0.10291858678955453, 0.0...","[0.5516667001143912, 0.053984627425796225, 0.0...","[0.08908466956992916, 0.08348552048004174, 0.2...","[0.25469104336831966, 0.17146641513977803, 0.1...","[0.6807481587027635, 0.2080916735234502, 0.064...","[0.1675329627325453, 0.400309056974854, 0.2538..."


In [61]:
comparsion_values
for index, row in comaprsion_df.iterrows():
    print(np.array(row))
    break

[0 -0.5831101237599363 -0.03932929737226567 -0.11601175922170077
 0.23024995347041788 1.0 -0.10965035693340158 -0.026310896738086852
 0.706911005032727 3.0707642122386076 -1.0579284895225902
 -0.22652434359352122 1.880901265779298 -0.22672185560580194]


In [55]:
a = np.array(query.iloc[:, 1:8])[0]
b = np.array(normalized_data.iloc[:, 1:8])[700]
a,b

(array([-0.8817296 , -0.95569794, -0.12193839,  0.36017435,  1.        ,
         0.91962453, -0.0251731 ]),
 array([-0.8817296 , -0.95569794, -0.12193839,  0.36017435,  1.        ,
         0.91962453, -0.0251731 ]))

In [56]:
from scipy.spatial import distance
distance.cosine(a,b)

0.0

In [16]:
comaprsion_df = normalized_data.copy()
for column in query.columns[1:8]:
    comaprsion_df[column] = normalized_data[column] - query[column][0]

for idx, column in enumerate(normalized_data.columns[8:]):
    comaprsion_df[column] = [standardize(wasserstein_distance(row, query[column][0]), idx) for row in normalized_data[column]]

In [18]:
comaprsion_df

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,0.298619,0.916369,0.005927,-0.129924,0.0,-1.029275,-0.001138,0.706911,3.070764,-1.057928,-0.226524,1.880901,-0.226722
1,1,0.240800,0.540376,0.003897,-0.087395,0.0,-0.822548,-0.001316,-0.202784,2.873449,-0.828880,0.420700,0.384816,-0.263937
2,2,0.278493,0.860025,0.003519,-0.079269,0.0,-0.962853,-0.001090,0.035338,2.795507,-0.351252,0.163516,0.921000,-0.775330
3,3,0.361112,1.348668,0.002045,-0.046862,0.0,-1.388371,-0.001225,-0.478161,3.147285,-0.917406,-0.361934,1.740425,-0.259657
4,4,0.157089,0.480123,0.026624,-0.474424,0.0,-1.225838,-0.001076,0.507394,2.988533,-0.355128,-0.083178,1.036771,-0.198301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,0.516702,0.944378,0.008904,-0.188938,0.0,-1.001538,-0.001314,-0.539549,3.298097,-0.675248,-0.387057,1.588114,0.908072
1810,1810,0.792147,1.769279,-0.016565,0.484318,0.0,-1.045580,-0.001278,-0.624279,3.193066,0.234825,0.118912,2.135730,-0.205266
1811,1811,0.371786,0.239518,0.075407,-0.932433,0.0,-1.323014,-0.001121,0.307365,2.919810,0.003639,-0.285808,-0.104959,-0.092936
1812,1812,1.294793,2.026358,-0.021057,0.659590,0.0,-1.225855,-0.001086,0.174479,3.067702,-1.141324,-0.664696,0.904675,0.120963


In [19]:
comaprsion_df = comaprsion_df.drop('id', axis=1)

In [28]:
comaprsion_df

,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0.298619,0.916369,0.005927,-0.129924,0.0,-1.029275,-0.001138,0.706911,3.070764,-1.057928,-0.226524,1.880901,-0.226722
1,0.240800,0.540376,0.003897,-0.087395,0.0,-0.822548,-0.001316,-0.202784,2.873449,-0.828880,0.420700,0.384816,-0.263937
2,0.278493,0.860025,0.003519,-0.079269,0.0,-0.962853,-0.001090,0.035338,2.795507,-0.351252,0.163516,0.921000,-0.775330
3,0.361112,1.348668,0.002045,-0.046862,0.0,-1.388371,-0.001225,-0.478161,3.147285,-0.917406,-0.361934,1.740425,-0.259657
4,0.157089,0.480123,0.026624,-0.474424,0.0,-1.225838,-0.001076,0.507394,2.988533,-0.355128,-0.083178,1.036771,-0.198301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,0.516702,0.944378,0.008904,-0.188938,0.0,-1.001538,-0.001314,-0.539549,3.298097,-0.675248,-0.387057,1.588114,0.908072
1810,0.792147,1.769279,-0.016565,0.484318,0.0,-1.045580,-0.001278,-0.624279,3.193066,0.234825,0.118912,2.135730,-0.205266
1811,0.371786,0.239518,0.075407,-0.932433,0.0,-1.323014,-0.001121,0.307365,2.919810,0.003639,-0.285808,-0.104959,-0.092936
1812,1.294793,2.026358,-0.021057,0.659590,0.0,-1.225855,-0.001086,0.174479,3.067702,-1.141324,-0.664696,0.904675,0.120963


In [31]:
comparsion_values = comaprsion_df.apply(np.linalg.norm, axis=1)
comparsion_values.sort_values(ascending=True)

718      1.546430
620      1.827039
1577     1.981575
1786     2.055974
697      2.077597
          ...    
704     20.131620
560     20.187788
1692    38.648202
1707    42.732196
166           NaN
Length: 1814, dtype: float64

In [32]:
comparsion_values = comaprsion_df.sum(axis = 1, skipna = True)
comparsion_values.sort_values(ascending=True)

687     -9.082920
700     -8.837959
722     -7.963649
1783    -6.811513
636     -6.527453
          ...    
560     25.688865
704     30.593658
84      31.410962
1692    35.885708
1707    44.011062
Length: 1814, dtype: float64

In [564]:
comaprsion_df.describe()

,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
count,1814.000000,1814.000000,1814.000000,1814.000000,1814.0,1814.000000,1813.000000,1814.000000,1814.000000,1814.000000,1814.000000,1814.000000,1814.000000
mean,0.588866,0.504440,0.005640,0.784274,0.0,0.403296,0.026324,1.215862,-0.187769,0.138782,0.202479,-0.132144,-0.280317
std,1.000276,1.000276,1.000276,1.000276,0.0,1.000276,1.000276,1.121622,0.585559,1.141851,1.027968,1.069507,0.761823
min,-0.350812,-0.451257,-0.191027,-0.822044,0.0,-1.478186,-0.000168,-1.841863,-1.509682,-1.636835,-1.503336,-1.649351,-1.289327
25%,-0.006678,-0.202018,-0.121928,0.152121,0.0,-0.188722,-0.000162,0.521575,-0.646236,-0.734160,-0.484667,-0.961155,-0.679493
50%,0.295789,0.132908,-0.093812,0.728173,0.0,0.242485,-0.000137,1.459475,-0.169690,-0.102883,-0.042313,-0.434048,-0.447976
75%,0.837997,0.921425,-0.031136,1.286046,0.0,0.792199,-0.000042,2.100986,0.232058,0.788649,0.594900,0.443569,-0.108679
max,19.325962,7.189479,38.288247,17.075289,0.0,13.815189,42.387499,3.074064,4.046008,4.631338,5.415124,3.514023,5.329263


In [566]:
normalized_data.describe()

,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity
count,1.814000e+03,1.814000e+03,1.814000e+03,1.814000e+03,1814.0,1.814000e+03,1.813000e+03
mean,-6.267191e-17,-5.483792e-17,-1.566798e-17,-1.175098e-16,1.0,3.133595e-17,-1.567662e-17
std,1.000276e+00,1.000276e+00,1.000276e+00,1.000276e+00,0.0,1.000276e+00,1.000276e+00
min,-9.396772e-01,-9.556979e-01,-1.966668e-01,-1.606318e+00,1.0,-1.881482e+00,-2.649182e-02
25%,-5.955436e-01,-7.064586e-01,-1.275672e-01,-6.321529e-01,1.0,-5.920179e-01,-2.648564e-02
50%,-2.930762e-01,-3.715324e-01,-9.945174e-02,-5.610093e-02,1.0,-1.608111e-01,-2.646072e-02
75%,2.491312e-01,4.169846e-01,-3.677571e-02,5.017720e-01,1.0,3.889033e-01,-2.636570e-02
max,1.873710e+01,6.685038e+00,3.828261e+01,1.629102e+01,1.0,1.341189e+01,4.236118e+01
